# CASE – ABSENTEÍSMO
## 01 – Exploração da Base de Eventos

Objetivo:
- Consolidar os 12 arquivos mensais
- Entender estrutura e qualidade dos dados
- Identificar inconsistências
- Definir regras iniciais de negócio

In [74]:
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option("display.max_columns", None)


In [75]:
path = "../banco_de_eventos/*.csv"

files = glob.glob(path)

df_eventos = pd.concat(
    [pd.read_csv(file, sep=";") for file in files],
    ignore_index=True
)

print("Total de registros:", df_eventos.shape[0])
df_eventos.head()


Total de registros: 12000


,id_funcionario,data_evento,ano_mes,tipo_evento,horas_evento,motivo,cid,origem,status
0,FUNC0614,08/01/2025,2025-01,Falta,4.0,Transporte,NaN,Gestor,Aprovado
1,FUNC0073,28/01/2025,2025-01,Falta,8.0,No-show,NaN,Sistema,Aprovado
2,FUNC0456,11/01/2025,2025-01,Atestado,8.0,Acidente,M54,Gestor,Aprovado
3,FUNC0440,12/01/2025,2025-01,Falta,8.0,No-show,NaN,Sistema,Aprovado
4,FUNC0486,12/01/2025,2025-01,Falta,8.0,No-show,NaN,Sistema,Aprovado


In [76]:
df_eventos.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id_funcionario  12000 non-null  object 
 1   data_evento     12000 non-null  object 
 2   ano_mes         12000 non-null  object 
 3   tipo_evento     12000 non-null  object 
 4   horas_evento    12000 non-null  float64
 5   motivo          10589 non-null  object 
 6   cid             1984 non-null   object 
 7   origem          12000 non-null  object 
 8   status          12000 non-null  object 
dtypes: float64(1), object(8)
memory usage: 843.9+ KB


In [77]:
df_eventos.describe(include="all")


,id_funcionario,data_evento,ano_mes,tipo_evento,horas_evento,motivo,cid,origem,status
count,12000,12000,12000,12000,12000.000000,10589,1984,12000,12000
unique,850,365,12,5,NaN,9,6,3,3
top,FUNC0156,16/07/2025,2025-01,Falta,NaN,No-show,J00,Sistema,Aprovado
freq,34,54,1000,7517,NaN,6350,345,10341,11200
mean,NaN,NaN,NaN,NaN,7.618162,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,4.577275,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,0.250000,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,5.650000,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,8.000000,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,8.000000,NaN,NaN,NaN,NaN


In [78]:
# Verificando se existem registros duplicados
duplicadas = df_eventos[df_eventos.duplicated(keep=False)]
duplicadas.head()

,id_funcionario,data_evento,ano_mes,tipo_evento,horas_evento,motivo,cid,origem,status
13,FUNC0205,10/01/2025,2025-01,Falta,8.0,No-show,NaN,Sistema,Aprovado
144,FUNC0502,29/01/2025,2025-01,Falta,8.0,No-show,NaN,Sistema,Aprovado
389,FUNC0577,27/01/2025,2025-01,Falta,8.0,No-show,NaN,Sistema,Aprovado
410,FUNC0660,22/01/2025,2025-01,Falta,8.0,No-show,NaN,Sistema,Aprovado
455,FUNC0205,10/01/2025,2025-01,Falta,8.0,No-show,NaN,Sistema,Aprovado


In [79]:
# Quantidade de registros duplicados
df_eventos.duplicated().sum()


np.int64(88)

In [80]:
# Deletando registros duplicados
df_eventos = df_eventos.drop_duplicates()
print("Total de registros após remoção de duplicados:", df_eventos.shape[0])

Total de registros após remoção de duplicados: 11912


In [81]:
# Verificando funcionário com maior carga naquele mês

df_horas = (
    df_eventos
        .groupby(["ano_mes", "id_funcionario"])["horas_evento"]
        .sum()
        .reset_index()
)

# df_horas.head()

idx = df_horas.groupby("ano_mes")["horas_evento"].idxmax()

df_maiores = df_horas.loc[idx].sort_values("ano_mes")

df_maiores


,ano_mes,id_funcionario,horas_evento
363,2025-01,FUNC0533,66.66
609,2025-02,FUNC0053,56.00
1631,2025-03,FUNC0729,64.00
2228,2025-04,FUNC0792,56.00
2566,2025-05,FUNC0431,76.00
2921,2025-06,FUNC0100,56.44
3456,2025-07,FUNC0045,67.53
4554,2025-08,FUNC0828,72.00
4815,2025-09,FUNC0384,96.00
5404,2025-10,FUNC0408,64.00


In [82]:
# Converter datas
df_eventos["data_evento"] = pd.to_datetime(df_eventos["data_evento"], dayfirst=True)
#df_eventos["ano_mes"] = pd.to_datetime(df_eventos["ano_mes"])

# Padronizar strings
df_eventos["tipo_evento"] = df_eventos["tipo_evento"].str.strip()
df_eventos["status"] = df_eventos["status"].str.strip()

# Preenchendo valores nulos na coluna "motivo" com "Nao informado"
df_eventos["motivo"] = df_eventos["motivo"].fillna("Nao informado")

df_eventos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11912 entries, 0 to 11999
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id_funcionario  11912 non-null  object        
 1   data_evento     11912 non-null  datetime64[ns]
 2   ano_mes         11912 non-null  object        
 3   tipo_evento     11912 non-null  object        
 4   horas_evento    11912 non-null  float64       
 5   motivo          11912 non-null  object        
 6   cid             1982 non-null   object        
 7   origem          11912 non-null  object        
 8   status          11912 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(7)
memory usage: 930.6+ KB


In [83]:
df_eventos.isnull().sum()


id_funcionario       0
data_evento          0
ano_mes              0
tipo_evento          0
horas_evento         0
motivo               0
cid               9930
origem               0
status               0
dtype: int64

In [84]:
# Verificar se as datas estão corretas

df_eventos["data_evento"] = pd.to_datetime(df_eventos["data_evento"], dayfirst=True)

df_eventos["ano_mes_data"] = df_eventos["data_evento"].dt.strftime("%Y-%m")

(df_eventos["ano_mes"] != df_eventos["ano_mes_data"]).sum()


np.int64(0)

In [85]:
# Analisar a distribuição dos tipos de eventos

df_eventos["tipo_evento"].value_counts()

tipo_evento
Falta        7434
Atestado     1982
Atraso       1122
Abono        1086
Suspensao     288
Name: count, dtype: int64

In [86]:
df_eventos["flag_evento_valido"] = (
    df_eventos["status"] == "Aprovado"
).astype(int)
df_eventos["flag_evento_valido"].value_counts()

flag_evento_valido
1    11112
0      800
Name: count, dtype: int64

In [87]:
df_eventos_agg = (
    df_eventos[df_eventos["status"] == "Aprovado"]
    .groupby(["id_funcionario", "ano_mes"])
    .agg(
        horas_evento_total=("horas_evento", "sum"),
        qtd_eventos=("horas_evento", "count")
    )
    .reset_index()
)
df_eventos_agg.head()

,id_funcionario,ano_mes,horas_evento_total,qtd_eventos
0,FUNC0001,2025-01,48.00,4
1,FUNC0001,2025-02,40.77,6
2,FUNC0001,2025-03,40.00,3
3,FUNC0001,2025-04,8.00,1
4,FUNC0001,2025-05,8.00,1


In [88]:
df_eventos.groupby("status")["horas_evento"].sum()


status
Aprovado    85545.08
Pendente     3251.24
Recusado     1944.34
Name: horas_evento, dtype: float64


# 📌 Conclusões da Base de Eventos

- 12.000 registros referentes a 2025.
- 88 registros duplicados.
- 850 funcionários distintos.
- 5 tipos de evento (Falta, Atestado, Atraso, Abono, Suspensão).
- Maior incidência: Falta (~62%).
- 94% das horas estão com status Aprovado.
- 11.112 eventos Aprovados.
- A métrica principal considerará todos os tipos de evento conforme regra do case.
- Apenas eventos com status "Aprovado" serão considerados.